In [14]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import os
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import math
import pickle
from PIL import Image, ImageEnhance, ImageOps, UnidentifiedImageError
import cv2
import numpy as np
import requests
from io import BytesIO
import ast
import torch
from torchvision import models, transforms
import asyncio
import aiohttp


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
csv_path = '/content/A2_Data.csv'

df = pd.read_csv(csv_path)
os.makedirs('reviews')
os.makedirs('processed_images')


# **Q1**

In [17]:
def preprocess_image(image_data):
    try:
        img = Image.open(BytesIO(image_data))

        # Apply preprocessing
        img = img.resize((256, 256))
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(1.2)
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(1.2)

        # Random flip
        if np.random.rand() > 0.5:
            img = ImageOps.flip(img)
        if np.random.rand() > 0.5:
            img = ImageOps.mirror(img)

        # Adjust exposure
        hsv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2HSV)
        hsv = np.array(hsv, dtype=np.float64)
        hsv[:, :, 2] = hsv[:, :, 2] * 1.3
        hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
        hsv = np.array(hsv, dtype=np.uint8)
        processed_img = Image.fromarray(cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB))

        return processed_img
    except Exception as e:
        print(f"Error processing image with ID: {e}")
        return None

async def fetch_image(session, url):
    async with session.get(url) as response:
        return await response.read()

async def preprocess_and_save_image(urls, serial):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_image(session, url) for url in urls]
        images = await asyncio.gather(*tasks)

        i = 1
        for image_data in images:
            processed_img = preprocess_image(image_data)
            if processed_img:

                processed_img.save(f"processed_images/{serial}_{i}.jpg")
                i += 1


tasks = []
for index, row in df.iterrows():
    try:
        urls = ast.literal_eval(str(row['Image']))
        tasks.append(preprocess_and_save_image(urls, row['Serial']))
    except Exception as e:
        print(f"Error processing row {row['Serial']}: {e}")


await asyncio.gather(*tasks)


Error processing image with ID: cannot identify image file <_io.BytesIO object at 0x7f33a758d210>
Error processing image with ID: cannot identify image file <_io.BytesIO object at 0x7f33a758d210>
Error processing image with ID: cannot identify image file <_io.BytesIO object at 0x7f33a836a1b0>
Error processing image with ID: cannot identify image file <_io.BytesIO object at 0x7f33a836a1b0>
Error processing image with ID: cannot identify image file <_io.BytesIO object at 0x7f33a77cad90>
Error processing image with ID: cannot identify image file <_io.BytesIO object at 0x7f3396e3a250>
Error processing image with ID: cannot identify image file <_io.BytesIO object at 0x7f3396e3a250>
Error processing image with ID: cannot identify image file <_io.BytesIO object at 0x7f3396e3a250>


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [18]:
model = models.resnet50(pretrained=True)
my_model = torch.nn.Sequential(*(list(model.children())[:-1]))
my_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [19]:
Matrix = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def feature_extraction(image_path, neural_net):
    image = Image.open(image_path).convert('RGB')
    preprocessed_image = Matrix(image)
    img_bat = torch.unsqueeze(preprocessed_image, 0)

    with torch.no_grad():
        features = neural_net(img_bat)
    return features.cpu().numpy().flatten()

extracted_features = {}

for image_file in os.listdir('processed_images'):
    if image_file.endswith('.jpg'):
        image_path = os.path.join('processed_images', image_file)
        features = feature_extraction(image_path, my_model)
        extracted_features[image_file] = features

with open('image_features.pkl', 'wb') as file:
    pickle.dump(extracted_features, file)





In [43]:
class TreeNode:
    def __init__(self, key):
        self.left = None
        self.right = None
        self.val = key


In [21]:
import pickle
import numpy as np

with open('image_features.pkl', 'rb') as f:
    features_dict = pickle.load(f)

normal_feat_dict = {}
for k, v in features_dict.items():
    if np.linalg.norm(v) > 0:
        normalized_vector = v / np.linalg.norm(v)
        normal_feat_dict[k] = normalized_vector
    else:
        normal_feat_dict[k] = v

with open('image_features_normalized.pkl', 'wb') as f:
    pickle.dump(normal_feat_dict, f)

In [23]:
pick = pd.read_pickle('image_features_normalized.pkl')
pick

{'3742_3.jpg': array([0.01007009, 0.01932949, 0.01128382, ..., 0.0005943 , 0.00165704,
        0.012548  ], dtype=float32),
 '2402_1.jpg': array([0.0762006 , 0.02375165, 0.01506543, ..., 0.01230874, 0.00942865,
        0.01022519], dtype=float32),
 '313_2.jpg': array([0.01990838, 0.01967477, 0.01147221, ..., 0.00854903, 0.00843185,
        0.01570192], dtype=float32),
 '415_1.jpg': array([0.01783619, 0.0072377 , 0.03250823, ..., 0.00599575, 0.0049316 ,
        0.01058102], dtype=float32),
 '2322_1.jpg': array([0.03140165, 0.0098065 , 0.00019827, ..., 0.0005497 , 0.02213359,
        0.02469633], dtype=float32),
 '1874_1.jpg': array([0.00707808, 0.01605238, 0.0029326 , ..., 0.01357454, 0.04087093,
        0.01229738], dtype=float32),
 '2713_5.jpg': array([0.00568792, 0.00966862, 0.02762777, ..., 0.01324169, 0.0148413 ,
        0.00785709], dtype=float32),
 '2713_2.jpg': array([0.02293442, 0.01886067, 0.04088385, ..., 0.02336801, 0.03110033,
        0.0256087 ], dtype=float32),
 '1462_1.j

In [24]:
with open('image_features_normalized.pkl', 'rb') as file:
    normalized_features = pickle.load(file)

# **Q2**

In [25]:
def process_text(data):
    data = data.lower()
    tokens = word_tokenize(data)
    custom_table = str.maketrans('', '', string.punctuation)
    stripped_tokens = [token.translate(custom_table) for token in tokens]
    stop_words = set(stopwords.words('english'))
    processed_words = [word for word in stripped_tokens if word not in stop_words and word.isalpha()]
    porter_stemmer = PorterStemmer()
    stemmed_words = [porter_stemmer.stem(word) for word in processed_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    return ' '.join(lemmatized_words)

result_folder = 'reviews'
os.makedirs(result_folder, exist_ok=True)

# Apply text processing
for idx, record in df.iterrows():
    text = process_text(str(record['Review Text']))
    file_location = os.path.join(result_folder, f"{record['Serial']}.txt")

    with open(file_location, 'w', encoding='utf-8') as file:
        file.write(text)


In [26]:
def calculate_tf(text_data):
    tokens = text_data.split()
    token_counts = {}
    for token in tokens:
        token_counts[token] = token_counts.get(token, 0) + 1
    total_tokens = len(tokens)
    tf = {token: count / total_tokens for token, count in token_counts.items()}
    return tf


def calculate_idf(documents):
    idf_counts = {}
    total_documents = len(documents)

    for document in documents:
        seen_tokens = set()
        for token in document.split():
            if token not in seen_tokens:
                idf_counts[token] = idf_counts.get(token, 0) + 1
                seen_tokens.add(token)

    idf = {}
    for token, count in idf_counts.items():
        idf[token] = math.log(total_documents / count)
    return idf


def calculate_tfidf(documents_data):
    inverse_document_frequency = calculate_idf(documents_data)
    tfidf_documents_list = []

    for document_data in documents_data:
        term_frequency = calculate_tf(document_data)
        tfidf = {}
        for token, tf_score in term_frequency.items():
          tfidf[token] = tf_score * inverse_document_frequency[token]
        tfidf_documents_list.append(tfidf)
    return tfidf_documents_list


def ids_tfidf(document_data, doc_ids):
    inverse_document_frequency = calculate_idf(document_data)
    tfidf_documents = {}
    for document, doc_id in zip(document_data, doc_ids):
        term_frequency = calculate_tf(document)
        tfidf = {}
        for token, tf_score in term_frequency.items():
          tfidf[token] = tf_score * inverse_document_frequency[token]
        tfidf_documents[doc_id] = tfidf
    return tfidf_documents


processed_texts = []
ids = []
data_folder = 'reviews'

for filename in sorted(os.listdir(data_folder)):
    if filename.endswith('.txt'):
        ids.append(filename.split('.')[0])
        file_path = os.path.join(data_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            processed_texts.append(file.read())

tfidf_results = ids_tfidf(processed_texts, ids)

with open('tfidf_scores_with_ids.pkl', 'wb') as f:
    pickle.dump(tfidf_results, f)


In [44]:
class BinaryTree:
    def __init__(self):
        self.root = None

    def insert(self, key):
        if self.root is None:
            self.root = TreeNode(key)
        else:
            self._insert_recursive(self.root, key)

    def _insert_recursive(self, current_node, key):
        if key < current_node.val:
            if current_node.left is None:
                current_node.left = TreeNode(key)
            else:
                self._insert_recursive(current_node.left, key)
        else:
            if current_node.right is None:
                current_node.right = TreeNode(key)
            else:
                self._insert_recursive(current_node.right, key)

    def inorder_traversal(self, node):
        if node:
            self.inorder_traversal(node.left)
            print(node.val, end=' ')
            self.inorder_traversal(node.right)

    def preorder_traversal(self, node):
        if node:
            print(node.val, end=' ')
            self.preorder_traversal(node.left)
            self.preorder_traversal(node.right)

    def postorder_traversal(self, node):
        if node:
            self.postorder_traversal(node.left)
            self.postorder_traversal(node.right)
            print(node.val, end=' ')


In [27]:
pick = pd.read_pickle('tfidf_scores_with_ids.pkl')
pick

{'100': {'aloha': 0.5756462732485114,
  'hawaii': 0.5756462732485114,
  'year': 0.22402063115050252,
  'fender': 0.460770092098434,
  'pbass': 0.9202434863103743,
  'new': 0.22778066742387498,
  'case': 0.21366248809403776,
  'fit': 0.14864927496490649,
  'like': 0.12542315809248814,
  'charm': 0.46012174315518717},
 '1004': {'great': 0.09495285693236373,
  'product': 0.178089221209706,
  'warranti': 0.6202306412408529,
  'usa': 0.1962721125685799,
  'purchas': 0.20159324243749976,
  'amazon': 0.12652309450309357,
  'hope': 0.15540422702642465,
  'never': 0.1274003198863366,
  'use': 0.04644756960824972,
  'amaz': 0.12921219955049237,
  'accord': 0.2878231366242557,
  'rode': 0.46012174315518717,
  'see': 0.09594104554141858,
  'screenshot': 0.2878231366242557,
  'microphon': 0.26823965207235,
  'amazoncom': 0.2878231366242557,
  'piti': 0.2878231366242557},
 '1005': {'excel': 0.13110646513702393,
  'microphon': 0.26823965207235,
  'background': 0.21316649207308674,
  'nois': 0.1421353

# **Q3**

In [28]:
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    similarity = dot_product / (norm_v1 * norm_v2)
    return similarity

## a)

In [29]:
def top3_Images(id, img_features, top_k=3):
    """Find the top_k most similar images to the input image."""
    # Extract the feature vector for the input image
    input_features = img_features[id]

    # Compute cosine similarity with all other images
    similarities = {}
    for iden, features in img_features.items():
        if iden != id:
            similarity = cosine_similarity(input_features, features)
            similarities[iden] = similarity

    # Sort by similarity
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)


    # most_similar_images = sorted_similarities[:top_k]

    # return most_similar_images

    # Filter to ensure unique base IDs
    all_ids = set(id.split('_')[0] for id in [id])
    top_3_images = []
    for image_id, similarity in sorted_similarities:
        first = image_id.split('_')[0]
        if first not in all_ids:
            all_ids.add(first)
            top_3_images.append((image_id, similarity))
            if len(top_3_images) == top_k:
                break
    return top_3_images


# Load image features
with open('image_features_normalized.pkl', 'rb') as f:
    image_features = pickle.load(f)


Image_Name = "4_1.jpg"
top_similar_images = top3_Images(Image_Name, image_features)
for id, similarity in top_similar_images:
    print(f"Image Name: {id}, Similarity: {similarity}")

Image Name: 2883_2.jpg, Similarity: 1.0
Image Name: 1466_1.jpg, Similarity: 0.8137103915214539
Image Name: 575_1.jpg, Similarity: 0.8040709495544434


## b)

In [30]:
with open('tfidf_scores_with_ids.pkl', 'rb') as f:
    pickle_tfidf = pickle.load(f)


def cosine_similarity_sparse(v1, v2):
    shared_keys = set(v1.keys()) & set(v2.keys())
    dot_product = sum([v1[x] * v2[x] for x in shared_keys])

    sum1 = sum([v1[x]**2 for x in v1.keys()])
    sum2 = sum([v2[x]**2 for x in v2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(dot_product) / denominator

def top3_reviews(id, pickle_tfidf, top_k=3):
    input_tfidf = pickle_tfidf[id]

    similarities = {}
    for review_id, tfidf in pickle_tfidf.items():
        if review_id != id:
            similarity = cosine_similarity_sparse(input_tfidf, tfidf)
            similarities[review_id] = similarity

    # Sort reviews by similarity
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    # Get the top_k similar review IDs
    most_similar_reviews = sorted_similarities[:top_k]

    return most_similar_reviews

review_id = '4'
most_similar_reviews = top3_reviews(review_id, pickle_tfidf)

for id, similarity in most_similar_reviews:
    print(f"Review ID: {id}, Similarity: {similarity}")

Review ID: 2883, Similarity: 0.9999999999999998
Review ID: 429, Similarity: 0.1724655546550282
Review ID: 3712, Similarity: 0.17103550209316773


In [31]:
review_corpus_sim, image_corpus_sim = {}, {}

for input_review_id in pickle_tfidf.keys():
    rws = top3_reviews(input_review_id, pickle_tfidf)
    review_corpus_sim[input_review_id] = rws

for input_id in image_features.keys():
    imgs = top3_Images(input_id, image_features)
    image_corpus_sim[input_id] = imgs


with open('review_only_retrieval.pkl', 'wb') as f:
    pickle.dump(review_corpus_sim, f)

with open('image_only_retrieval.pkl', 'wb') as f:
    pickle.dump(image_corpus_sim, f)


# **Q4**

In [33]:
def avg_similarities(id, image_features, tfidf_scores):
    # image similarities
    img_similarities = {}
    img_key = id + "_1.jpg"
    primary_image = image_features[img_key]
    for id1, features in image_features.items():
        if id1 != img_key:
            similar = cosine_similarity(primary_image, features)
            img_similarities[id1.split('_')[0]] = similar

    # review similarities
    review_similarities = {}
    primary_image_tfidf = tfidf_scores[id]
    for id1, tfidf in tfidf_scores.items():
        if id1 != id:
            sim = cosine_similarity_sparse(primary_image_tfidf, tfidf)
            review_similarities[id1] = sim

    # average similarities
    dict_avg_similarities = {}
    for id1 in set(img_similarities.keys()).union(review_similarities.keys()):
        avg_sim = (img_similarities.get(id1, 0) + review_similarities.get(id1, 0)) / 2
        dict_avg_similarities[id1] = avg_sim
    # show top threee
    sorted_similarities = sorted(dict_avg_similarities.items(), key=lambda x: x[1], reverse=True)[:3]

    return sorted_similarities

Serial = '4'
sorted_similarities = avg_similarities(Serial, image_features, pickle_tfidf)

for id, sim in sorted_similarities:
    print(f"ID: {id}, Average Similarity: {sim}")

composite_similarities_all = {}
for Serial in image_features.keys():
    x = Serial.split('_')[0]
    top3_similar = avg_similarities(x, image_features, pickle_tfidf)

    composite_similarities_all[x] = top3_similar

# Save the results
with open('composite_retrieval.pkl', 'wb') as f:
    pickle.dump(composite_similarities_all, f)

# print(composite_similarities_all.keys())
print(composite_similarities_all['4'])

ID: 2883, Average Similarity: 0.9999999999999999
ID: 3533, Average Similarity: 0.41442079917752467
ID: 3580, Average Similarity: 0.4134874523308613
[('2883', 0.9999999999999999), ('3533', 0.41442079917752467), ('3580', 0.4134874523308613)]


# **Q5**

In [34]:
image_url = 'https://images-na.ssl-images-amazon.com/images/I/71raKRNMOPL._SY88.jpg'
input_review = '''Awesome stand!

Tip: The bottom part that supports the guitar had a weird angle when arrived, making the guitar slide back, becoming almost 100% on a vertical.
To solve this, I assembled the product and the put a some pressure on the support frame, making it bend a little. Now my guitar sits perfectly. Check photos!'''


In [35]:
image_serial = (df[df['Image'].astype(str).str.contains(image_url)])['Serial'].unique().tolist()
review_serial = (df[df['Review Text'] == input_review])['Serial'].unique().tolist()
image_serial, review_serial

([1714], [1714])

In [36]:
url_id = str(image_serial[0]) + "_1.jpg"
review_id = str(review_serial[0])
print(url_id, review_id)

1714_1.jpg 1714


In [46]:
if __name__ == "__main__":
    # Create a binary tree instance
    bt = BinaryTree()

    # Insert nodes
    bt.insert(3)
    bt.insert(1)
    bt.insert(4)
    bt.insert(2)

    # print("In-order traversal:")
    # bt.inorder_traversal(bt.root)
    # print("\nPre-order traversal:")
    # bt.preorder_traversal(bt.root)
    # print("\nPost-order traversal:")
    # bt.postorder_traversal(bt.root)


In [37]:
image_retrieved_similarities = pd.read_pickle('image_only_retrieval.pkl')

print(f"UNLEASHING THE POWER OF IMAGES [Input Image ID: {url_id}]: \n\n")

input_code = url_id.split('_')[0]

for retrieved_id, similarity_score in image_retrieved_similarities[url_id]:
    print("--------------------------------------------------------------------------------------------")
    x = retrieved_id.split('_')[0]

    image_urls = df[df['Serial'] == int(x)]['Image'].values
    reviews = df[df['Serial'] == int(x)]['Review Text'].values
    print("Retrieved Image ID: ", retrieved_id)

    # Print each URL individually
    for url in image_urls:
        print("Image URL(S): ", url)

    # Print each review individually
    for review in reviews:
        print("Review: ", review)

    print(f"Cosine Similarity of Images: {similarity_score:.4f}")

    # Finding text similarity between input_code and x
    text_similarity = cosine_similarity_sparse(pickle_tfidf[input_code], pickle_tfidf[str(x)])
    print(f"Cosine similarity of text (between {input_code} & {str(x)}): {text_similarity:.4f}")

    composite_similarity = (similarity_score + text_similarity) / 2
    print(f"Composite Similarity Score: {composite_similarity:.4f}")
    print("--------------------------------------------------------------------------------------------")

    print('\n\n')


UNLEASHING THE POWER OF IMAGES [Input Image ID: 1714_1.jpg]: 


--------------------------------------------------------------------------------------------
Retrieved Image ID:  1819_4.jpg
Image URL(S):  ['https://images-na.ssl-images-amazon.com/images/I/61KdaAUshwL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71zriquspkL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/61lAMAeamzL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/61QOfHtEI5L._SY88.jpg']
Review:  kinda flimsy but oit does the job
Cosine Similarity of Images: 0.8093
Cosine similarity of text (between 1714 & 1819): 0.0000
Composite Similarity Score: 0.4046
--------------------------------------------------------------------------------------------



--------------------------------------------------------------------------------------------
Retrieved Image ID:  306_1.jpg
Image URL(S):  ['https://images-na.ssl-images-amazon.com/images/I/71G-FwZ-zdL._SY88.jpg']
Review:  this pedal

In [38]:
retrieved_text_similarities = pd.read_pickle('review_only_retrieval.pkl')

input_text_id = '1205'
print(f"UNLEASHING TEXTUAL EXPLORATION [Input Text ID: {input_text_id}]: ")

for retrieved_id, similarity_score in retrieved_text_similarities[input_text_id]:
    print("-" * 50)
    image_urls = df[df['Serial'] == int(retrieved_id)]['Image'].values
    reviews = df[df['Serial'] == int(retrieved_id)]['Review Text'].values

    print("Retrieved Text ID: ", retrieved_id)
    for url in image_urls:
        print("Image URL(S): ", url)
    for review in reviews:
        print("Review: ", review)

    image_similarity = cosine_similarity(image_features[str(input_text_id) + "_1.jpg"], image_features[str(retrieved_id) + "_1.jpg"])
    print(f"Cosine Similarity of Images (between {input_text_id} & {str(retrieved_id)}): {image_similarity:.4f}")

    print(f"Cosine similarity of textual content: {similarity_score:.4f}")

    composite_similarity = (similarity_score + image_similarity) / 2
    print(f"Composite Similarity Score: {composite_similarity:.4f}")
    print("-" * 50)

    print('\n\n')


UNLEASHING TEXTUAL EXPLORATION [Input Text ID: 1205]: 
--------------------------------------------------
Retrieved Text ID:  3828
Image URL(S):  ['https://images-na.ssl-images-amazon.com/images/I/71vT2-nW7-L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71Wz9iild8L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71Z5NiWe6TL._SY88.jpg']
Review:  The picture doesn't do this bridge justice, I was skeptical of the quality when I ordered this Tremolo bridge but was pleasantly surprised when it arrived. I wasn't so concerned with the quality because I planed on using it on an old broken down imported Kramer ST300 that I refinished. I gave it a 3 because of the screws it takes a lot of hand strength to get the screws seated. You have to be very careful that you don't strip the philip screw heads the screw driver doesn't quit fit the number 2. I'll take the guitar to my shared work shop to see if a larger screw driver fits better.
Cosine Similarity of Images (bet

As we can see image based retrival shows more accuracy than image